# Relational Data Querying Using LLMs

This notebook demonstrates how relational data can be queried in natural language using LLMs. 

### Use Case
We have a relational database with multiple tables. We want to ask questions in natural language, get them translated into SQL, and then executed. The final result is a relational table.  

### Prototype: Approach and Data
The solution approach is to fetch the dataschema first, and then provide it as a contex for generating SQL based on a natural language query. We implement this using a standard SQL querying chain provided by Langchain. 

We use the `classicmodel` database [1]. The database contains a set of tables which is typical for order management systems. We use only two tables: orders and order details (line items). We assume an external MySQL instance with `classicmodel` database loaded into it.

### Usage and Productization
This is a basic prototype that demonstrates the concept. Productization would typically require to pre-process the schema to select only the relevant tables, handle SQL execution errors, and post-process the results to elect the optimal representation such as a line plot, bar chart, or textual summary.

### References:
1. https://www.mysqltutorial.org/mysql-sample-database.aspx (copy available in `tensor-house-data/search/orders-database`)

## Environment Setup and Initialization

In [217]:
#
# Initialize LLM provider
# (google-cloud-aiplatform must be installed)
#
from google.cloud import aiplatform
aiplatform.init(
    project='<< specify your project name here >>',
    location='us-central1'
)

#
# Initialize DB connection
#
from langchain.llms import VertexAI
from langchain import SQLDatabase, SQLDatabaseChain

db = SQLDatabase.from_uri("mysql+pymysql://root:@localhost/classicmodels", include_tables=['orders', 'orderdetails'], sample_rows_in_table_info=0)

In [218]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

Determine which tables need to be joined before generating the SQL query. Limit the number of rows in the SQL result by {top_k}.

Question: {input}"""
custom_prompt = PromptTemplate(
    input_variables=["input", "table_info", "dialect", "top_k"], template=_DEFAULT_TEMPLATE
)

llm = VertexAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, 
                                     db, 
                                     top_k=3,
                                     verbose=True, 
                                     prompt=custom_prompt,
                                     return_intermediate_steps=True)

## Example Queries

In [222]:
#
# Convert the input query into SQL and execute it - basic example
#
result = db_chain("How many orders are there?")



> Entering new  chain...
How many orders are there?
SQLQuery:SELECT COUNT(*) FROM orders
SQLResult: [(326,)]
Answer:326
> Finished chain.


In [224]:
#
# Convert the input query into SQL and execute it - more complex examples
#
result = db_chain("What was the revenue for each date?")



> Entering new  chain...
What was the revenue for each date?
SQLQuery:SELECT SUM(quantityOrdered * priceEach) AS revenue, orderDate FROM orderdetails JOIN orders ON orderdetails.orderNumber = orders.orderNumber GROUP BY orderDate ORDER BY orderDate LIMIT 3
SQLResult: [(Decimal('10223.83'), datetime.date(2003, 1, 6)), (Decimal('10549.01'), datetime.date(2003, 1, 9)), (Decimal('5494.78'), datetime.date(2003, 1, 10))]
Answer:[(10223.83, datetime.date(2003, 1, 6)), (10549.01, datetime.date(2003, 1, 9)), (5494.78, datetime.date(2003, 1, 10))]
> Finished chain.


In [221]:
#
# Print the table info used in the query
#
print(result["intermediate_steps"][0]['table_info'])


CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)DEFAULT CHARSET=latin1 ENGINE=InnoDB


CREATE TABLE orders (
	`orderNumber` INTEGER NOT NULL, 
	`orderDate` DATE NOT NULL, 
	`requiredDate` DATE NOT NULL, 
	`shippedDate` DATE, 
	status VARCHAR(15) NOT NULL, 
	comments TEXT, 
	`customerNumber` INTEGER NOT NULL, 
	PRIMARY KEY (`orderNumber`), 
	CONSTRAINT orders_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)DEFAULT CHARSET=latin1 ENGINE=InnoDB
